In [1]:
%pip install "plotly" "nbformat>=4.2.0" "numpy" "kaleido"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
queries = ['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11','Q12','Q13','Q14']
approaches = ['OCPQ','DuckDB','SQLite','Kuzu'] # , 'SQL'

In [2]:
import json
import numpy as np
res = {}
for q in queries:
    res[q] = {}
    for a in approaches:
        try:
            with open(f"./{q}/{a.lower()}-res.json") as f:
                res[q][a] = json.load(f)
        except:
                res[q][a] = False

In [3]:
import plotly.graph_objects as go
import plotly.colors as colors_sq
import numpy as np


CELL_WIDTH_PX  = 140 
CELL_HEIGHT_PX = 70    
MIN_W, MIN_H   = 1200, 600
MAX_W, MAX_H   = 12000, 8000
PNG_SCALE      = 2      


fig_width  = min(MAX_W, max(MIN_W, CELL_WIDTH_PX  * len(queries)))
fig_height = min(MAX_H, max(MIN_H, CELL_HEIGHT_PX * len(approaches)))

fig = go.Figure(data=go.Heatmap(
    z=[[None if ((res[q][a] == False) or (type(res[q][a]) == str)) else 1000 * np.mean(res[q][a]) for q in queries] for a in approaches],
    x=queries,
    y=[f"<b>{a}</b>" if a == "OCPQ" else a for a in approaches],
    text=[
        [
            "" if ((res[q][a] == False) or (type(res[q][a]) == str)) else
            "" if np.isnan(np.mean(res[q][a])) else f"{np.round(1000 * np.mean(res[q][a]),0):.0f}ms"
            for q in queries
        ]
        for a in approaches
    ],
    texttemplate="%{text}",
    textfont={"size": 20},  
    colorscale=colors_sq.sequential.YlGn_r,
    colorbar=dict(title=dict(text="Duration (ms)", side="bottom", font=dict(size=16))),
    xgap=1, 
    ygap=1
))

fig.update_yaxes(ticksuffix=" ", title="Approach", showgrid=True, tickson='boundaries')
fig.update_xaxes(title="Query", showgrid=True, tickson='boundaries', tickangle=45)

fig.update_layout(
    font=dict(size=24),
    autosize=False,
    width=fig_width,    
    height=fig_height,  
    template="plotly_white",
    title="",
    margin=dict(l=80, r=40, t=40, b=120) 
)

for q_i in range(len(queries)):
    for a_i in range(len(approaches)):
        q = queries[q_i]
        a = approaches[a_i]
        if res[q][a] == False or type(res[q][a]) == str:
            fig.add_annotation(
                x=q_i, y=a_i,
                text="✔" if res[q][a] == True else "✗" if res[q][a] == False else res[q][a],
                showarrow=False,
                font=dict(size=36, color="green" if res[q][a] == True else "lightgray" if res[q][a] == False else "gray")
            )

fig.show()


fig.write_image("heatmapJustin.svg", width=fig_width, height=fig_height)
fig.write_image("heatmapJustin.pdf", width=fig_width, height=fig_height)
fig.write_image("heatmapJustin.png", width=fig_width, height=fig_height, scale=PNG_SCALE)  
